Use ATENT as attack on pretrained CIFAR10 WRN34-10 ATENT model [Table 6 of paper]

Notebook contains printed result from evaluation of pretrained model

### IMPORT LIBRARIES

In [1]:
import sys,os
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
import random

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.backends.cudnn as cudnn

from torchvision import transforms, datasets

import numpy as np

### IMPORT UTILITIES

In [2]:
from functional import entropySmoothing

### SET TRAINING PARAMETERS

In [3]:
args = {}
#data loading
args['seed'] = 1
args['test_batch_size'] = 128
args['train_batch_size'] = 128
kwargs = {'num_workers': 4, 'pin_memory': True}
args['no_cuda'] = False

if not args['no_cuda']:
    if torch.cuda.is_available():
        DEVICE = 'cuda'
    else:
        DEVICE = 'cpu'
else:
    DEVICE = 'cpu'

# params for SGLD/PGD attack (inner loop)
args['attacktype'] = 'ATENT' #['PGD','ATENT']
args['attack'] = 'l_inf'
args['norm'] = 'inf'
args['epsilon'] = 0.031
args['num_steps'] = 20
args['step_size'] = 0.003
args['random'] =True

### LOAD DATA

In [4]:
dataset = 'CIFAR10' # [MNIST, CIFAR10]

# setup data loader
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform_train, download=True)
val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform_test, download=True)
    
train_loader = DataLoader(train, batch_size=args['test_batch_size'], shuffle=True, **kwargs)
val_loader = DataLoader(val, batch_size=args['train_batch_size'], shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


### LOAD NETWORK

In [5]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    Net = WideResNet
    NetName = 'WideResNet34'

### SET RANDOM SEED 

In [6]:
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(args['seed'])
    cudnn.benchmark = True
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])

### WHITEBOX L-INF ATTACK

In [7]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args['epsilon'],
                  num_steps=args['num_steps'],
                  step_size=args['step_size']
                 ):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args['random']:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(DEVICE)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    with torch.no_grad():
        loss_pgd = nn.CrossEntropyLoss()(model(X_pgd), y)
    return err, err_pgd, loss_pgd.item()

def eval_adv_test_whitebox(model, device, test_loader,attacktype):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0
    lossrob = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        if attacktype == 'ATENT':
            err_natural, err_robust, losspgd = _atent_whitebox(model, X, y)
        elif attacktype == 'PGD':
            err_natural, err_robust, losspgd = _pgd_whitebox(model, X, y)        
        robust_err_total += err_robust
        natural_err_total += err_natural
        lossrob = lossrob + losspgd
    rob = 100-100*robust_err_total.item()/len(test_loader.dataset)   
    lossrob /= len(test_loader)
    print('robust test loss:',lossrob)
    print('natural_acc_total: ', 100-100*natural_err_total.item()/len(test_loader.dataset))
    print('robust_acc_total: ', rob)    
    return rob

In [8]:
def _atent_whitebox(model,
                  x,
                  y,
                  eps=args['epsilon'],
                  L=args['num_steps'],
                  step=args['step_size'],
                  norm = args['norm']
                   ):
    # Adversial perturbation
    alpha=0.9
    loss = 0
    out = model(x)    
    err = (out.data.max(1)[1] != y.data).float().sum()
    loss_fn = nn.CrossEntropyLoss()
    
    for l in range(L):     
        
        if l==0: # initialize using random perturbation of true x, run for one epoch
            random=True
            xp = None
            projector=False
        elif l>0 and l<L-1: # initialize with previous iterate of adversarial perturbation, run one epoch
            random=False
            xp=x_adv
            projector = False
        elif l == L-1: # initialize with previous iterate, run one epoch, project to epsilon ball
            random=False
            xp = x_adv
            projector=True
            
        x_adv,bfl = entropySmoothing(model, x, y, loss_fn, xp=xp, step=step, eps=eps, norm=norm, random=random, ep=1e-3,projector=projector)
        loss = (1-alpha)*loss + alpha*loss_fn(out, y)
        
    err_pgd = (model(x_adv).data.max(1)[1] != y.data).float().sum()
        
    return err, err_pgd, loss.item()

### LOAD FROM CHECKPOINT

In [9]:
model_ATENT = Net().to(DEVICE)
model_ATENT = nn.DataParallel(model_ATENT)
#load state dict here
pathstr = '../trainedmodels/CIFAR10/BEST_model-nn-epoch76-robacc57.pt
model_ATENT.load_state_dict(torch.load(pathstr))

<All keys matched successfully>

In [10]:
print('Running '+args['attacktype']+' as attack ... ')
rob = eval_adv_test_whitebox(model_ATENT, DEVICE, val_loader,args['attacktype'])            

Running ATENT as attack ... 
robust test loss: 0.5295534843130957
natural_acc_total:  85.67
robust_acc_total:  60.5
